HW#3 Causal AI - Group Memebers: Aman Pathak & Aseel Al-Omary

#### Recap on our Causal question from last assignment was:

What is the causal effect of improved healthcare access (treatment) on diabetes-related health outcomes (outcome), controlling for socioeconomic factors (e.g., income, education), clinical factors (e.g., BMI, blood pressure), and the mediating effects of lifestyle factors (e.g., physical activity, diet) 


###  AIPW applied to SDoH and Diabetes Risk
To estimate the causal effect of improved healthcare access on diabetes-related health outcomes while accounting for confounding factors, the Augmented Inverse Probability Weighting (AIPW) method is well-suited. 
Here's how and why AIPW fits this context and how it will be implemented:

#### Why AIPW is Suitable
- Robustness to Model Misspecification: 
AIPW provides consistent estimates of the causal effect as long as either the outcome model (predicting diabetes-related health outcomes) or the propensity score model (predicting healthcare access) is correctly specified.

- Handling High-Dimensional Covariates: 
Given the inclusion of various socioeconomic, clinical, and lifestyle factors, AIPW is advantageous as it can efficiently adjust for these covariates, reducing confounding bias. It allows us to control for potential confounders that might simultaneously influence healthcare access and diabetes outcomes.

- Flexibility with Heterogeneous Effects: 
AIPW can accommodate continuous and binary outcomes, making it adaptable to different diabetes-related measures

### How AIPW is implemented to our Data
- 1- Start by importing dependancies (libraries)
- 2- Reading the data to workspace
- 3- Defining: treatment, outcome, and covariates (regressors) and preparing them through (Dummy Variable Handling for categorical features and Feature Scaling for numerical features)
- 5- Splitting 80% training - 20%testing
- 6- DRLearner model inetiation, training and iterative model fitting and evaluation (The DRLearner model is trained and evaluated separately for each dummy treatment variable), to be able to set up the causal forest.
- 7-Estimating the Average Treatment Effect (ATE)

### Importing libraries

In [2]:
#importing libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from econml.dr import DRLearner
import warnings

### Importing Data

In [3]:
# Reading Data
data = pd.read_csv("combined_X_y.csv")

Handling Missingness

In [4]:
# Check for missing values
if data.isnull().sum().any():
    print("There are missing values in the dataset.")
else:
    print("No missing values found in the dataset.")

No missing values found in the dataset.


Managing datatypes and encoding categorical data preparing for later modeling, to avoid the dummy variable trap

In [5]:
# numerical Variables/columns
num_cols = ["BMI"]
# categorical Variables/columns
cat_cols = ['Stroke','MentHlth','Education','GenHlth',
 'Fruits','PhysActivity','Smoker','DiffWalk','CholCheck',
 'PhysHlth','HighBP','Sex', 'HvyAlcoholConsump','Age',
 'NoDocbcCost', 'HeartDiseaseorAttack','Income','HighChol',
 'Veggies']
# Encode categorical columns
xf = pd.get_dummies(data[cat_cols])
data = pd.concat([data.drop(cat_cols, axis=1), xf], axis=1)

### Define Regressors 

Regressors are used used in both the propensity score model and the outcome regression model

In [6]:
regressors = num_cols

In [7]:
# Add in the dummy names to the list of regressors
cat_var_dummy_names = list(xf.columns)
regressors = regressors + cat_var_dummy_names

 Define: treatment, outcome, and covariates

In [9]:
# Defining treatment, outcome, and covariates for our dataset as per to the causal Question
treatment = "AnyHealthcare"
outcome = "Diabetes_binary"
covariates = ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']

Define transformed variables

In [10]:
# Define transformed variables using regressors (covarietes after dummies)
X = data[regressors]
T = data[treatment]  # Ensure this matches the actual column name exactly
Y = data[outcome]

### Modeling

#### Data splitting: creating training and test datasets
- X: This is the feature matrix, which contains the independent predictors excluding the treatment and outcome.
- T: This is the treatment variable (healthcare access)
- Y: This is the outcome variable, which contains the dependent variable being studied (Diabetes Risk).

In [14]:
# Split data
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X, T, Y, test_size=0.2, random_state=42)

In [15]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### VIF (Variance Inflation Factor)

an optional step: due to multiple warnings we decided to check for Multicollinearity as highly correlated covariates can cause issues with the co-variance matrix. 
the check for multicollinearity utilized VIF (Variance Inflation Factor) 
Generally:
- A VIF of 1 indicates no correlation between the given predictor and other predictors.
- A VIF between 1 and 5 suggests moderate correlation but typically not enough to warrant corrective measures.
- A VIF above 5 indicates a high correlation and suggests significant multicollinearity.
- A VIF above 10 is often seen as problematic and suggests multicollinearity.

In [16]:
# Check for multicollinearity using VIF (optional step)
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print("VIF:", vif_data)

VIF:                  feature        VIF
0                    BMI  17.772375
1                 Stroke   1.126698
2               MentHlth   1.462138
3              Education  28.107251
4                GenHlth  10.655296
5                 Fruits   3.032425
6           PhysActivity   4.627034
7                 Smoker   1.931259
8               DiffWalk   1.838034
9              CholCheck  21.879450
10              PhysHlth   1.999503
11                HighBP   2.298462
12                   Sex   1.910027
13     HvyAlcoholConsump   1.083521
14                   Age   9.500974
15           NoDocbcCost   1.188276
16  HeartDiseaseorAttack   1.289697
17                Income  13.830783
18              HighChol   2.029529
19               Veggies   5.821505


Using the above guideline, the features with VIFs above 10 and indicative of high multicollinearity are:
(Education, BMI, CholCheck, GenHlth, Income, Age)
- Education: VIF = 28.107251
- BMI: VIF = 17.772375
- CholCheck: VIF = 21.879450
- GenHlth: VIF = 10.655296
- Income: VIF = 13.830783
- Age: VIF = 9.500974 (borderline high)

Given these results,addressing multicollinearity could include:removing highly correlated features if removing one or any of the features is feasible without losing significant information.

### Scenario_A: Modeling and calculating AIPW without addressing multicollinearity

In [17]:
# Define models
propensity_model = LogisticRegression(max_iter=1000)
outcome_model = RandomForestClassifier()  # Use classifier for binary outcomes

In [20]:
# Initialize DRLearner with discrete_outcome explicitly set to True
aipw_model = DRLearner(model_propensity=propensity_model, 
                       model_regression=outcome_model,
                       discrete_outcome=True)

In [21]:
# fit model again
aipw_model.fit(Y_train, T_train, X=X_train_scaled)


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [22]:
# Evaluate model performance
ate = aipw_model.ate(X_test_scaled)
ate_conf_int = aipw_model.ate_interval(X_test_scaled)
print("Estimated ATE:", ate)
print("95% Confidence Interval:", ate_conf_int)

Estimated ATE: [-0.00680782]
95% Confidence Interval: (-0.06364846485851786, 0.05003282830570859)


## Inference Results:

a. Estimated ATE: [-0.00680782]
This means that, on average, individuals with improved healthcare access have a slightly lower probability of developing diabetes compared to those without improved access. The negative value indicates a decrease in the outcome variable (diabetes) due to the treatment (improved healthcare access).
 
b. 95% Confidence Interval: (-0.06364846485851786, 0.05003282830570859)
This represents the interval within which the true average treatment effect is likely to lie with 95% confidence. As it includes zero, we cannot definitively conclude that there is a statistically significant effect of healthcare access on diabetes risk at the 5% significance level
 
c. Interpretation:
As the estimated ATE is negative, it does suggest a potential benefit of healthcare access, but its magnitude is very very low. The narrow confidence interval indicates moderate certainty in the estimate.
 
d. Assumption:
Overlap: The treatment and control groups must share sufficient similarity in observed characteristics. This ensures that the propensity score model can accurately estimate treatment probabilities for everyone.


### Scenario_B: Modeling and calculating AIPW addressing multicollinearity by removing highly correlated features

In [23]:
high_vif_features = ['BMI', 'CholCheck', 'Education', 'GenHlth', 'Income']
revised_columns = [col for col in X.columns if col not in high_vif_features]

# Define new features excluding the high VIF features
X2 = data[revised_columns]
T2 = data[treatment]  # Use multiple binary columns as treatment
Y2 = data[outcome]

In [24]:
# Split data
X_train2, X_test2, T_train2, T_test2, Y_train2, Y_test2 = train_test_split(X2, T2, Y2, test_size=0.2, random_state=42)

In [25]:
# Scale features
scaler = StandardScaler()
X_train2_scaled = scaler.fit_transform(X_train2)
X_test2_scaled = scaler.transform(X_test2)

In [26]:
# Define models
propensity_model2 = LogisticRegression(max_iter=1000)
outcome_model2 = RandomForestClassifier()  # Use classifier for binary outcomes

In [27]:
# Initialize DRLearner with discrete_outcome explicitly set to True
aipw_model2 = DRLearner(model_propensity=propensity_model2, 
                       model_regression=outcome_model2,
                       discrete_outcome=True)

In [28]:
# fit model again
aipw_model2.fit(Y_train2, T_train2, X=X_train2_scaled)


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [32]:
# Evaluate model performance
ate2 = aipw_model2.ate(X_test2_scaled)
ate_conf_int2 = aipw_model2.ate_interval(X_test2_scaled)
print("Estimated ATE:", ate2)
print("95% Confidence Interval:", ate_conf_int2)

Estimated ATE: [-0.01620563]
95% Confidence Interval: (-0.05919128508383329, 0.026780019906768857)


The Estimated ATE for both scinarios is close without removing= [-0.00680782] , with removing= [-0.01620563] , possible reasons might indicate that the negative effect of the treatment on the outcome is slightly stronger when multicollinearity issues are addressed.

The difference in ATE estimates between Scenario A and Scenario B, while present, is relatively small. This suggests that multicollinearity might not have had a huge impact on the treatment effect estimation in this case, but addressing it does seem to produce a more stable and slightly adjusted estimate. It is generally advisable to address multicollinearity to ensure the robustness and reliability of causal inference analyses.